House Price Prediction

1 .Collecting Data

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import concurrent.futures
import random
from urllib.parse import urljoin

# Cấu hình
START_PAGE = 1
END_PAGE = 1300
MAX_WORKERS = 10  # Số luồng chạy cùng lúc
OUTPUT_FILE = 'batdongsan_raw_data.csv'

# Headers giả lập trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://batdongsan.vn/'
}

# Các hàm xử lý dữ liệu
def extract_price(price_text):
    if not price_text: return None
    price_text = price_text.lower().strip().replace('tỷ', 't').replace('tỉ', 't').replace('triệu', 'm').replace(' ', '')
    total_price = 0
    if 't' in price_text:
        parts = price_text.split('t')
        if parts[0]:
            try: total_price += float(parts[0].replace(',', '.')) * 1e9
            except: pass
        if len(parts) > 1 and 'm' in parts[1]:
            try: total_price += float(parts[1].split('m')[0].replace(',', '.')) * 1e6
            except: pass
    elif 'm' in price_text:
        try: total_price += float(price_text.split('m')[0].replace(',', '.')) * 1e6
        except: pass
    return total_price / 1e9 if total_price > 0 else None

def extract_area(area_text):
    if not area_text: return None
    try:
        match = re.search(r'([\d\.\,]+)', area_text)
        if match: return float(match.group(1).replace(',', '.'))
    except: pass
    return None

def extract_bedrooms(text):
    if not text: return None
    try:
        match = re.search(r'(\d+)\s*Phòng ngủ', text)
        if match: return int(match.group(1))
    except: pass
    return None

def extract_toilets(text):
    if not text: return None
    try:
        match = re.search(r'(\d+)\s*WC', text)
        if match: return int(match.group(1))
    except: pass
    return None

def crawl_page(page_num, session):
    """
    Hàm crawl một trang đơn lẻ, sử dụng session chung
    """
    if page_num == 1:
        url = "https://batdongsan.vn/ban-nha-ho-chi-minh"
    else:
        url = f"https://batdongsan.vn/ban-nha-ho-chi-minh/p{page_num}"

    try:
        # Random sleep nhẹ để giảm tải server
        time.sleep(random.uniform(0.5, 1.5))

        response = session.get(url, headers=headers, timeout=20)

        if response.status_code != 200:
            print(f"❌ Trang {page_num}: Lỗi Status {response.status_code}")
            return []

        soup = BeautifulSoup(response.text, 'html.parser')

        # Logic tìm thẻ card
        property_cards = soup.find_all('a', class_='card-cm')
        if not property_cards:
            property_cards = soup.find_all('div', class_='card-container')
            if property_cards:
                property_cards = property_cards[0].find_all('a', class_='card-cm')

        page_data = []
        for card in property_cards:
            try:
                # Trích xuất thông tin cơ bản
                title_elem = card.find('h3', class_='title')
                title = title_elem.text.strip() if title_elem else ""

                detail_url = card.get('href', '')
                if detail_url and not detail_url.startswith('http'):
                    detail_url = urljoin('https://batdongsan.vn', detail_url)

                location_elem = card.find('div', class_='description')
                location_text = location_elem.text.strip() if location_elem else ""
                district = location_text.split(',')[0].strip() if location_text else ""

                area_elem = card.find('div', class_='description-item')
                area_text = area_elem.text.strip() if area_elem else ""

                bedroom_text = card.find(text=re.compile(r'\d+\s*Phòng ngủ'))
                toilet_text = card.find(text=re.compile(r'\d+\s*WC'))

                price_elem = card.find('div', class_='price')
                price_raw = price_elem.text.strip() if price_elem else ""

                time_elem = card.find('div', class_='time')
                post_time = time_elem.text.strip() if time_elem else ""

                # Gom vào dict
                item = {
                    'Page': page_num,
                    'Title': title,
                    'Price_Raw': price_raw,
                    'Price_Billion': extract_price(price_raw),
                    'District': district,
                    'Area_m2': extract_area(area_text),
                    'Bedrooms': extract_bedrooms(bedroom_text),
                    'Toilets': extract_toilets(toilet_text),
                    'Address': location_text,
                    'Post_Time': post_time,
                    'Link': detail_url
                }
                page_data.append(item)
            except Exception:
                continue

        print(f"✅ Trang {page_num}: Lấy được {len(page_data)} tin")
        return page_data

    except Exception as e:
        print(f"⚠️ Trang {page_num} lỗi: {str(e)[:50]}")
        return []

def main():
    print("="*50)
    print(f"🚀 BẮT ĐẦU CRAWL DATA THÔ: {START_PAGE} -> {END_PAGE}")
    print(f"⚡ Số luồng (Threads): {MAX_WORKERS}")
    print("="*50)

    all_results = []
    start_time = time.time()

    # Tạo Session để tái sử dụng kết nối (Tăng tốc độ)
    session = requests.Session()

    # Sử dụng ThreadPoolExecutor để chạy đa luồng
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Tạo danh sách các tác vụ cần làm (future objects)
        future_to_page = {
            executor.submit(crawl_page, page, session): page
            for page in range(START_PAGE, END_PAGE + 1)
        }

        # Xử lý kết quả khi từng luồng hoàn thành
        completed_count = 0
        total_tasks = len(future_to_page)

        for future in concurrent.futures.as_completed(future_to_page):
            page = future_to_page[future]
            try:
                data = future.result()
                if data:
                    all_results.extend(data)
            except Exception as exc:
                print(f"❌ Trang {page} sinh ra ngoại lệ: {exc}")

            completed_count += 1
            # Cập nhật tiến độ mỗi 50 trang
            if completed_count % 50 == 0:
                print(f"📊 Tiến độ: {completed_count}/{total_tasks} trang. Tổng tin: {len(all_results)}")

    # Lưu kết quả
    print("\n💾 Đang lưu dữ liệu vào CSV...")
    df = pd.DataFrame(all_results)

    if not df.empty:
        # Sắp xếp lại cột cho dễ nhìn
        cols = ['Page', 'Title', 'Price_Raw', 'Price_Billion', 'Area_m2', 'District', 'Address', 'Bedrooms', 'Toilets', 'Post_Time', 'Link']
        # Chỉ lấy các cột có trong df (đề phòng trường hợp lỗi)
        existing_cols = [c for c in cols if c in df.columns]
        df = df[existing_cols]

        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"✅ HOÀN THÀNH! File đã lưu: {OUTPUT_FILE}")
        print(f"📈 Tổng số dòng: {len(df)}")
    else:
        print("❌ Không thu thập được dữ liệu nào.")

    print(f"⏱️ Tổng thời gian chạy: {int(time.time() - start_time)} giây")

if __name__ == "__main__":
    main()

🚀 BẮT ĐẦU CRAWL DATA THÔ: 1 -> 1300
⚡ Số luồng (Threads): 10
✅ Trang 9: Lấy được 24 tin
✅ Trang 6: Lấy được 24 tin
✅ Trang 2: Lấy được 24 tin
✅ Trang 10: Lấy được 24 tin
✅ Trang 3: Lấy được 24 tin
✅ Trang 7: Lấy được 24 tin
✅ Trang 1: Lấy được 24 tin
✅ Trang 8: Lấy được 24 tin
✅ Trang 4: Lấy được 24 tin
✅ Trang 5: Lấy được 24 tin
✅ Trang 12: Lấy được 24 tin
✅ Trang 13: Lấy được 24 tin
✅ Trang 19: Lấy được 24 tin
✅ Trang 17: Lấy được 24 tin
✅ Trang 11: Lấy được 24 tin
✅ Trang 20: Lấy được 24 tin
✅ Trang 16: Lấy được 24 tin
✅ Trang 14: Lấy được 24 tin
✅ Trang 15: Lấy được 24 tin
✅ Trang 18: Lấy được 24 tin
✅ Trang 21: Lấy được 24 tin
✅ Trang 22: Lấy được 24 tin
✅ Trang 24: Lấy được 24 tin
✅ Trang 25: Lấy được 24 tin
✅ Trang 27: Lấy được 24 tin
✅ Trang 28: Lấy được 24 tin
✅ Trang 23: Lấy được 24 tin
✅ Trang 30: Lấy được 24 tin
✅ Trang 26: Lấy được 24 tin
✅ Trang 31: Lấy được 24 tin
✅ Trang 29: Lấy được 24 tin
✅ Trang 34: Lấy được 24 tin
✅ Trang 35: Lấy được 24 tin
✅ Trang 32: Lấy được 24 

Dánh id cho mỗi cột

In [2]:
import pandas as pd

# 1. Đọc file CSV hiện tại
input_file = 'batdongsan_raw_data.csv' # Tên file cũ của bạn
output_file = 'batdongsan_data_with_id.csv' # Tên file mới sẽ lưu

print("⏳ Đang đọc dữ liệu...")
df = pd.read_csv(input_file)

# 2. Thêm cột ID
# range(1, len(df) + 1): Tạo dãy số từ 1 đến hết bảng
# df.insert(0, ...): Chèn vào vị trí đầu tiên (cột số 0)
df.insert(0, 'id', range(1, len(df) + 1))

# 3. Lưu ra file mới
print(f"💾 Đang lưu file mới: {output_file}...")
df.to_csv(output_file, index=False, encoding='utf-8-sig')

# 4. Kiểm tra kết quả
print("✅ Đã xong! Preview 5 dòng đầu:")
print(df[['id', 'Title', 'Price_Raw']].head())

⏳ Đang đọc dữ liệu...
💾 Đang lưu file mới: batdongsan_data_with_id.csv...
✅ Đã xong! Preview 5 dòng đầu:
   id                                              Title       Price_Raw
0   1  SIÊU PHẨM GÓC 2 MẶT TIỀN – TRƯƠNG THỊ NHƯ, XUÂ...  7 tỷ 990 triệu
1   2  BÁN GẤP NHÀ MẶT TIỀN HIỆP THÀNH 17 – TRUNG TÂM...  7 tỷ 890 triệu
2   3  HOT – HÀNG HIẾM GIỮA QUẬN 12 BÁN XƯỞNG HẺM XE ...           20 tỷ
3   4  Bán nhà mặt tiền 50x120m giá 95 tỷ Đườn...           95 tỷ
4   5  Bán nhà mặt tiền 23x99m giá 58 tỷ Đường...           58 tỷ


thêm trường description

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time
import random
import os

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_with_id.csv'  # File CSV đã có cột ID và Link
OUTPUT_FILE = 'batdongsan_full_desc.csv'    # File kết quả đầu ra
MAX_WORKERS = 8      # Số luồng (Để 8 là vừa phải, server vẫn chịu tốt)
SAVE_BATCH = 500     # Cứ xong 500 link thì lưu file 1 lần (để lỡ lỗi không mất hết)

# Headers giả lập trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://batdongsan.vn/'
}

def get_description(url, session):
    """
    Hàm vào link chi tiết lấy description từ thẻ <div id="more1">
    """
    if not url or pd.isna(url):
        return ""

    try:
        # Ngủ ngẫu nhiên 0.5 - 1.5 giây để giảm tải server
        time.sleep(random.uniform(0.5, 1.5))

        response = session.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return "Lỗi truy cập"

        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Tìm thẻ div có id="more1" (Chứa full nội dung ẩn)
        desc_div = soup.find('div', id='more1')

        # Fallback: Nếu không thấy more1, thử tìm more (nội dung hiển thị rút gọn)
        if not desc_div:
            desc_div = soup.find('div', id='more')

        if desc_div:
            # get_text(separator='\n') sẽ thay thế thẻ <br> và </p> bằng xuống dòng
            # strip() để cắt khoảng trắng đầu đuôi
            content = desc_div.get_text(separator='\n').strip()

            # Xử lý thêm: Xóa các dòng trống quá nhiều
            lines = [line.strip() for line in content.split('\n') if line.strip()]
            return '\n'.join(lines)

        return "Không có mô tả"

    except Exception as e:
        return f"Lỗi code: {str(e)}"

def process_row(row, session):
    """Hàm xử lý cho từng dòng trong dataframe"""
    row_id = row['id']
    url = row['Link']
    desc = get_description(url, session)
    return row_id, desc

def main():
    print("⏳ Đang đọc file dữ liệu...")
    try:
        df = pd.read_csv(INPUT_FILE)
        print(f"📊 Tổng số dòng cần crawl: {len(df)}")
    except FileNotFoundError:
        print("❌ Không tìm thấy file input. Hãy chắc chắn tên file đúng.")
        return

    # Kiểm tra xem đã crawl dở chưa (để chạy tiếp thay vì chạy lại từ đầu)
    processed_ids = set()
    if os.path.exists(OUTPUT_FILE):
        df_done = pd.read_csv(OUTPUT_FILE)
        if 'id' in df_done.columns:
            processed_ids = set(df_done['id'])
            print(f"🔄 Đã tìm thấy file cũ, tiếp tục crawl từ dòng thứ {len(processed_ids)}...")

    # Lọc ra những dòng chưa crawl
    # Chỉ lấy các dòng có ID chưa nằm trong processed_ids
    df_to_process = df[~df['id'].isin(processed_ids)]

    if df_to_process.empty:
        print("✅ Đã crawl xong hết rồi!")
        return

    print(f"🚀 Bắt đầu crawl {len(df_to_process)} dòng còn lại...")
    print(f"⚡ Số luồng: {MAX_WORKERS}")

    session = requests.Session()
    results = []
    count = 0

    # Chuyển DataFrame thành list dict để dễ xử lý
    rows_to_process = df_to_process.to_dict('records')

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit các task
        future_to_row = {executor.submit(process_row, row, session): row for row in rows_to_process}

        for future in concurrent.futures.as_completed(future_to_row):
            try:
                r_id, desc = future.result()
                results.append({'id': r_id, 'Description': desc})
                count += 1

                # In tiến độ
                if count % 10 == 0:
                    print(f"✅ Đã xử lý: {count} tin", end='\r')

                # Cứ mỗi SAVE_BATCH (500) tin thì lưu file một lần
                if count % SAVE_BATCH == 0 or count == len(rows_to_process):
                    # Tạo dataframe từ kết quả mới
                    temp_df = pd.DataFrame(results)

                    # Nếu file chưa tồn tại thì ghi mới (kèm header)
                    # Nếu file đã tồn tại thì ghi nối tiếp (không ghi header)
                    header_mode = not os.path.exists(OUTPUT_FILE)
                    temp_df.to_csv(OUTPUT_FILE, mode='a', index=False, header=header_mode, encoding='utf-8-sig')

                    print(f"\n💾 Đã lưu checkpoint ({count} dòng mới)...")
                    results = [] # Reset buffer sau khi lưu

            except Exception as exc:
                print(f"\n❌ Có lỗi ngoại lệ: {exc}")

    # Lưu nốt những dòng còn lại trong buffer (nếu có)
    if results:
        temp_df = pd.DataFrame(results)
        header_mode = not os.path.exists(OUTPUT_FILE)
        temp_df.to_csv(OUTPUT_FILE, mode='a', index=False, header=header_mode, encoding='utf-8-sig')
        print("\n✅ ĐÃ LƯU XONG TOÀN BỘ!")

    print("\n🔗 BƯỚC CUỐI: GHÉP DỮ LIỆU")
    # Đọc file gốc và file description vừa crawl để ghép lại
    df_org = pd.read_csv(INPUT_FILE)
    df_desc = pd.read_csv(OUTPUT_FILE)

    # Merge dựa trên cột id
    df_final = pd.merge(df_org, df_desc, on='id', how='left')

    final_filename = 'batdongsan_data_FINAL.csv'
    df_final.to_csv(final_filename, index=False, encoding='utf-8-sig')
    print(f"🎉 FILE CUỐI CÙNG CỦA BẠN: {final_filename}")

if __name__ == "__main__":
    main()

✅ Đã xử lý: 2000 tin
💾 Đã lưu checkpoint (2000 dòng mới)...
✅ Đã xử lý: 2500 tin
💾 Đã lưu checkpoint (2500 dòng mới)...
✅ Đã xử lý: 3000 tin
💾 Đã lưu checkpoint (3000 dòng mới)...
✅ Đã xử lý: 3500 tin
💾 Đã lưu checkpoint (3500 dòng mới)...
✅ Đã xử lý: 4000 tin
💾 Đã lưu checkpoint (4000 dòng mới)...
✅ Đã xử lý: 4500 tin
💾 Đã lưu checkpoint (4500 dòng mới)...
✅ Đã xử lý: 5000 tin
💾 Đã lưu checkpoint (5000 dòng mới)...
✅ Đã xử lý: 5500 tin
💾 Đã lưu checkpoint (5500 dòng mới)...
✅ Đã xử lý: 6000 tin
💾 Đã lưu checkpoint (6000 dòng mới)...
✅ Đã xử lý: 6500 tin
💾 Đã lưu checkpoint (6500 dòng mới)...
✅ Đã xử lý: 7000 tin
💾 Đã lưu checkpoint (7000 dòng mới)...
✅ Đã xử lý: 7500 tin
💾 Đã lưu checkpoint (7500 dòng mới)...
✅ Đã xử lý: 8000 tin
💾 Đã lưu checkpoint (8000 dòng mới)...
✅ Đã xử lý: 8500 tin
💾 Đã lưu checkpoint (8500 dòng mới)...
✅ Đã xử lý: 9000 tin
💾 Đã lưu checkpoint (9000 dòng mới)...
✅ Đã xử lý: 9500 tin
💾 Đã lưu checkpoint (9500 dòng mới)...
✅ Đã xử lý: 10000 tin
💾 Đã lưu checkpoin

LỌC DATA THÊM THƯỜNG Price_Per_m2, Is_MatTien, Width_m

In [1]:
import pandas as pd
import re

# --- CẤU HÌNH TÊN FILE ---
INPUT_FILE = 'batdongsan_data_FINAL.csv'  # Đặt tên file csv của bạn là data.csv
OUTPUT_FILE = 'batdongsan_data_CLEAN.csv'

# --- HÀM XỬ LÝ ---
def extract_width(row):
    # Gộp & Làm sạch sơ bộ
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()

    # --- BƯỚC CHUẨN HÓA QUAN TRỌNG ---
    text = text.replace(',', '.')           # Đổi phẩy thành chấm (6,5 -> 6.5)
    text = re.sub(r'[❌*×]', 'x', text)     # Đổi mọi icon nhân thành 'x'
    text = re.sub(r'[–\-_:]', ' ', text)    # Đổi gạch nối dính chữ thành khoảng trắng

    # MẪU 1: Cụm "Chiều rộng... 77m" (Xử lý lỗi typo)
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10 # Fix lỗi 77m -> 7.7m
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Từ khóa "Ngang" + từ đệm (khủng/đẹp) + Số
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: Dạng kích thước A x B (Có chữ m hoặc không)
    # Tìm tất cả các cặp số dạng AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2) # Lấy số nhỏ làm chiều ngang

                # Ưu tiên: Nếu dòng đó có chữ 'm' (vd: 6x13m) thì tin luôn
                if 'm' in m.group(0) and 2 <= width < 50:
                    return width
                # Nếu không có 'm' (vd: 4x16), lưu lại xem xét sau
                if 2 <= width < 50:
                    temp_width = width
            except: continue

        # Nếu đã tìm thấy dạng AxB hợp lý
        if 'temp_width' in locals(): return temp_width

    return None

def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()

    # 1. DANH SÁCH TỪ KHÓA LOẠI TRỪ (Thấy mấy từ này là biết ko phải mặt tiền xịn)
    # Ví dụ: "sát mặt tiền", "cách mặt tiền", "gần mặt tiền", "hẻm xe hơi quay đầu"
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']

    for kw in exclude_keywords:
        if kw in text:
            return 0 # Gặp từ 'sát'/'cách' -> Trả về 0 ngay lập tức (Là hẻm)

    # 2. DANH SÁCH TỪ KHÓA NHẬN DIỆN MẶT TIỀN (Sau khi đã lọc bọn 'sát/gần')
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']

    for kw in positive_keywords:
        if kw in text:
            return 1 # Có mặt tiền

    return 0 # Không thấy gì -> Mặc định là hẻm

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý dữ liệu...")

try:
    # 1. Đọc file với đúng các cột bạn cung cấp
    df = pd.read_csv(INPUT_FILE)

    # 2. Xử lý chiều ngang (Width_m)
    df['Width_m'] = df.apply(extract_width, axis=1)

    # 3. Xử lý Mặt tiền (Is_MatTien)
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # 4. Tính Đơn giá (Price_Per_m2) = (Tỷ * 1000) / m2
    # Chỉ tính khi Diện tích > 0 để tránh lỗi chia cho 0
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if x['Area_m2'] > 0 else 0,
        axis=1
    )

    # 5. Lưu file
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"Xong! File kết quả là: {OUTPUT_FILE}")
    print("-" * 30)
    # In thử 5 dòng các cột quan trọng để kiểm tra
    print(df[['id', 'Title', 'Width_m', 'Is_MatTien', 'Price_Per_m2']].head().to_string())

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{INPUT_FILE}'. Hãy chắc chắn file nằm cùng thư mục với code.")

Đang xử lý dữ liệu...
Xong! File kết quả là: batdongsan_data_CLEAN.csv
------------------------------
   id                                                                   Title  Width_m  Is_MatTien  Price_Per_m2
0   1       SIÊU PHẨM GÓC 2 MẶT TIỀN – TRƯƠNG THỊ NHƯ, XUÂN THỚI SƠN, HÓC MÔN     14.0           1         28.95
1   2   BÁN GẤP NHÀ MẶT TIỀN HIỆP THÀNH 17 – TRUNG TÂM QUẬN 12 – GIÁ CHỐT MỀM      5.0           1         82.19
2   3  HOT – HÀNG HIẾM GIỮA QUẬN 12 BÁN XƯỞNG HẺM XE TẢI 8M, P. TÂN THỚI NHẤT     14.5           0         60.06
3   4              Bán nhà mặt tiền 50x120m giá 95 tỷ Đường Thanh Niên      NaN           1         15.83
4   5            Bán nhà mặt tiền 23x99m giá 58 tỷ Đường Nhị Bình 15     23.5           1         25.47


Xuất data mà 1 trong các trường Toilets,Bedrooms,  Price_Billion,Price_Per_m2 ,Area_m2  bị thiếu

In [13]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_CLEAN_FULL_CALCULATED.csv'  # File dữ liệu sạch từ bước trước
OUTPUT_FILE = 'data_thieu_thong_tin.csv' # File chứa các dòng bị lỗi

try:
    print(f"Đang đọc file {INPUT_FILE}...")
    df = pd.read_csv(INPUT_FILE)

    # Danh sách các cột cần kiểm tra
    check_cols = ['Address']

    # Lọc các dòng có ít nhất 1 ô bị Null trong danh sách cột trên
    # how='any' nghĩa là chỉ cần 1 cột bị thiếu là lấy ra
    df_missing = df[df[check_cols].isnull().any(axis=1)]

    # Lưu ra file mới
    df_missing.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print(f"XỬ LÝ XONG!")
    print(f"Tổng số dòng bị thiếu thông tin: {len(df_missing)} dòng")
    print(f"Đã xuất dữ liệu ra file: {OUTPUT_FILE}")
    print("-" * 40)

    # In thống kê chi tiết lại một lần nữa cho tập dữ liệu này
    print("Chi tiết số lượng thiếu trong file này:")
    print(df_missing[check_cols].isnull().sum())

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{INPUT_FILE}'. Hãy chắc chắn bạn đã chạy code tạo file data_clean_final.csv ở bước trước.")
except KeyError as e:
    print(f"Lỗi: File thiếu cột {e}. Hãy kiểm tra lại tên cột trong file CSV.")

Đang đọc file batdongsan_data_CLEAN_FULL_CALCULATED.csv...
----------------------------------------
XỬ LÝ XONG!
Tổng số dòng bị thiếu thông tin: 0 dòng
Đã xuất dữ liệu ra file: data_thieu_thong_tin.csv
----------------------------------------
Chi tiết số lượng thiếu trong file này:
Address    0
dtype: int64


Dựa kết quả lọc width tìm lại trường Area_m2 bị trống, tính lại số tiền trên mỗi m2

In [6]:
import pandas as pd
import re
import numpy as np

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_CLEAN.csv'
OUTPUT_FILE = 'batdongsan_data_CLEAN_FULL_CALCULATED.csv'

# ================= 1. GIỮ NGUYÊN HÀM CỦA BẠN (Đã tối ưu) =================
def extract_width(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    # MẪU 1: Chiều rộng...
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Ngang...
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2)
                if 'm' in m.group(0) and 2 <= width < 50: return width
                if 2 <= width < 50: temp_width = width
            except: continue
        if 'temp_width' in locals(): return temp_width
    return None

def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']
    for kw in exclude_keywords:
        if kw in text: return 0
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']
    for kw in positive_keywords:
        if kw in text: return 1
    return 0

# ================= 2. HÀM MỚI: TÌM CHIỀU DÀI & TÍNH DIỆN TÍCH =================
def recover_missing_area(row):
    # Nếu Diện tích đã có dữ liệu hợp lệ (>0) thì giữ nguyên, không cần tính lại
    current_area = row.get('Area_m2', 0)
    if pd.notnull(current_area) and current_area > 0:
        return current_area

    # Nếu không có Chiều Ngang (Width), thì chịu thua, không tính được Area
    width = row.get('Width_m')
    if pd.isnull(width) or width == 0:
        return current_area # Trả về như cũ (Null hoặc 0)

    # --- BẮT ĐẦU ĐI TÌM CHIỀU DÀI (LENGTH) ---
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    length = None

    # MẪU A: Tìm từ khóa "Dài..." (Ví dụ: Ngang 5 dài 20)
    match_dai = re.search(r'dài\s*(\d+[.]?\d*)', text)
    if match_dai:
        try:
            val = float(match_dai.group(1))
            # Chiều dài thường phải lớn hơn chiều rộng và < 100m (nhà phố)
            if val >= width and val < 150:
                length = val
        except: pass

    # MẪU B: Tìm trong cụm AxB (Ví dụ: 5x20)
    # Logic: Nếu Width khớp với 1 trong 2 số, thì số kia là Length
    if length is None:
        matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                # Nếu 1 trong 2 số xấp xỉ bằng Width mà mình đã tìm ra
                # (Cho sai số 0.1 phòng trường hợp làm tròn)
                if abs(n1 - width) < 0.1: length = n2
                elif abs(n2 - width) < 0.1: length = n1

                if length: break
            except: continue

    # --- TÍNH TOÁN DIỆN TÍCH ---
    if length and length > 0:
        calculated_area = width * length
        return round(calculated_area, 2)

    return current_area # Không tìm thấy Length thì trả về Null như cũ

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý dữ liệu...")
try:
    df = pd.read_csv(INPUT_FILE)

    # Bước 1: Trích xuất Width (như cũ)
    print("- Đang trích xuất chiều ngang...")
    df['Width_m'] = df.apply(extract_width, axis=1)

    # Bước 2: CỨU DỮ LIỆU AREA (Bước mới)
    # Lấp đầy các ô Area bị trống bằng cách tính Width * Length
    print("- Đang tính toán để lấp đầy Area bị thiếu...")
    df['Area_m2'] = df.apply(recover_missing_area, axis=1)

    # Bước 3: Xác định mặt tiền
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # Bước 4: Tính lại đơn giá (Sau khi đã cứu được Area)
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if pd.notnull(x['Area_m2']) and x['Area_m2'] > 0 else 0,
        axis=1
    )

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"Xong! File kết quả: {OUTPUT_FILE}")
    print("-" * 30)

    # In kiểm tra những dòng mà Area được cứu sống (trước đó là null/0 nhưng giờ có Width*Length)
    # Logic in: Có Width, Có Area, nhưng Price_Raw có thể check sau
    print("Mẫu 5 dòng dữ liệu sau khi tính toán:")
    print(df[['id', 'Width_m', 'Area_m2', 'Price_Per_m2']].head(10).to_string())

except Exception as e:
    print(f"Lỗi: {e}")

Đang xử lý dữ liệu...
- Đang trích xuất chiều ngang...
- Đang tính toán để lấp đầy Area bị thiếu...
Xong! File kết quả: batdongsan_data_CLEAN_FULL_CALCULATED.csv
------------------------------
Mẫu 5 dòng dữ liệu sau khi tính toán:
   id  Width_m  Area_m2  Price_Per_m2
0   1     14.0    276.0         28.95
1   2      5.0     96.0         82.19
2   3     14.5    333.0         60.06
3   4      NaN   6000.0         15.83
4   5     23.5   2277.0         25.47
5   6      NaN     86.0         84.88
6   7      4.0     76.0         98.42
7   8      5.0     85.0         70.59
8   9      5.0     50.0        106.00
9  10      NaN     24.0        127.08


Trích xuất bổ sung thêm tường Floors cải tiến code trên

In [4]:
import pandas as pd
import re
import numpy as np

# --- CẤU HÌNH ---
# Hãy đảm bảo tên file INPUT đúng với file bạn đang có
INPUT_FILE = 'batdongsan_data_CLEAN.csv'
OUTPUT_FILE = 'batdongsan_data_CLEAN_FULL_CALCULATED_CAITIEN.csv'

# ================= 1. HÀM TRÍCH XUẤT CHIỀU NGANG (WIDTH) =================
def extract_width(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    # MẪU 1: Chiều rộng...
    match = re.search(r'chiều\s*rộng\s*(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if val > 50: val = val / 10
            if 0 < val < 50: return val
        except: pass

    # MẪU 2: Ngang...
    match = re.search(r'(?:ngang|rộng|mt)(?:.{0,15}?)\s+(\d+[.]?\d*)', text)
    if match:
        try:
            val = float(match.group(1))
            if 0 < val < 50: return val
        except: pass

    # MẪU 3: AxB
    matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
    if matches:
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                width = min(n1, n2)
                if 'm' in m.group(0) and 2 <= width < 50: return width
                if 2 <= width < 50: temp_width = width
            except: continue
        if 'temp_width' in locals(): return temp_width
    return None

# ================= 2. HÀM XÁC ĐỊNH MẶT TIỀN =================
def check_mattien(row):
    text = str(row['Title']) + " " + str(row['Description'])
    text = text.lower()
    exclude_keywords = ['sát mặt tiền', 'cách mặt tiền', 'gần mặt tiền', 'sau mặt tiền']
    for kw in exclude_keywords:
        if kw in text: return 0
    positive_keywords = ['mặt tiền', 'mtkd', 'mt đường', 'lô góc', '2 mặt tiền']
    for kw in positive_keywords:
        if kw in text: return 1
    return 0

# ================= 3. HÀM TÍNH TOÁN LẠI DIỆN TÍCH =================
def recover_missing_area(row):
    current_area = row.get('Area_m2', 0)
    if pd.notnull(current_area) and current_area > 0:
        return current_area

    width = row.get('Width_m')
    if pd.isnull(width) or width == 0:
        return current_area

    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace(',', '.')
    text = re.sub(r'[❌*×]', 'x', text)
    text = re.sub(r'[–\-_:]', ' ', text)

    length = None
    # MẪU A: Tìm từ khóa "Dài..."
    match_dai = re.search(r'dài\s*(\d+[.]?\d*)', text)
    if match_dai:
        try:
            val = float(match_dai.group(1))
            if val >= width and val < 150:
                length = val
        except: pass

    # MẪU B: Tìm trong cụm AxB
    if length is None:
        matches = list(re.finditer(r'(\d+[.]?\d*)\s*m?\s*x\s*(\d+[.]?\d*)', text))
        for m in matches:
            try:
                n1 = float(m.group(1))
                n2 = float(m.group(2))
                if abs(n1 - width) < 0.1: length = n2
                elif abs(n2 - width) < 0.1: length = n1
                if length: break
            except: continue

    if length and length > 0:
        return round(width * length, 2)
    return current_area

# ================= 4. HÀM TRÍCH XUẤT SỐ TẦNG (FLOORS) - MỚI! =================
def extract_floors(row):
    text = str(row.get('Description', '')) + " " + str(row.get('Title', ''))
    text = text.lower()
    text = text.replace('tấm', 'tầng') # Chuẩn hóa

    floors = 0
    found_structure = False

    # 4.1 Chiến thuật cộng dồn: Trệt + Lầu + Lửng + Sân thượng
    matches_lau = re.findall(r'(\d+)\s*lầu', text)
    if matches_lau:
        num_lau = max([float(x) for x in matches_lau])
        floors = num_lau + 1 # +1 cho tầng trệt
        found_structure = True
    elif 'trệt' in text and 'lầu' in text:
        floors = 2 # Mặc định 1 trệt 1 lầu
        found_structure = True

    if found_structure:
        if 'lửng' in text: floors += 0.5
        if 'sân thượng' in text or 'chuồng cu' in text or 'tum' in text: floors += 0.5
        return floors

    # 4.2 Chiến thuật tìm số tổng: "3 tầng", "4 tấm"
    match_tang = re.search(r'(\d+)\s*tầng', text)
    if match_tang:
        return float(match_tang.group(1))

    # 4.3 Các trường hợp đặc biệt
    if 'cấp 4' in text or 'cap 4' in text: return 1.0
    if 'gác' in text: return 1.5

    return 1.0 # Mặc định bèo nhất là 1 tầng

# ================= CHẠY CHƯƠNG TRÌNH =================
print("Đang xử lý dữ liệu...")
try:
    df = pd.read_csv(INPUT_FILE)

    # Bước 1: Trích xuất Width
    print("- Đang trích xuất chiều ngang (Width)...")
    df['Width_m'] = df.apply(extract_width, axis=1)

    # Bước 2: CỨU DỮ LIỆU AREA
    print("- Đang tính toán để lấp đầy Area bị thiếu...")
    df['Area_m2'] = df.apply(recover_missing_area, axis=1)

    # Bước 3: Xác định mặt tiền
    print("- Đang xác định Mặt tiền (Is_MatTien)...")
    df['Is_MatTien'] = df.apply(check_mattien, axis=1)

    # Bước 4: Trích xuất số tầng (MỚI)
    print("- Đang đọc kết cấu số tầng (Floors)...")
    df['Floors'] = df.apply(extract_floors, axis=1)

    # Bước 5: Tính lại đơn giá
    print("- Đang cập nhật đơn giá (Price_Per_m2)...")
    df['Price_Per_m2'] = df.apply(
        lambda x: round((x['Price_Billion'] * 1000) / x['Area_m2'], 2)
        if pd.notnull(x['Area_m2']) and x['Area_m2'] > 0 else 0,
        axis=1
    )

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 30)
    print(f"✅ XONG! File kết quả đầy đủ: {OUTPUT_FILE}")
    print("-" * 30)

    # In kiểm tra mẫu
    print("Mẫu 5 dòng dữ liệu (Width, Area, Floors, Price/m2):")
    print(df[['id', 'Width_m', 'Area_m2', 'Floors', 'Price_Per_m2']].head(10).to_string())

except Exception as e:
    print(f"❌ Lỗi: {e}")

Đang xử lý dữ liệu...
- Đang trích xuất chiều ngang (Width)...
- Đang tính toán để lấp đầy Area bị thiếu...
- Đang xác định Mặt tiền (Is_MatTien)...
- Đang đọc kết cấu số tầng (Floors)...
- Đang cập nhật đơn giá (Price_Per_m2)...
------------------------------
✅ XONG! File kết quả đầy đủ: batdongsan_data_CLEAN_FULL_CALCULATED_CAITIEN.csv
------------------------------
Mẫu 5 dòng dữ liệu (Width, Area, Floors, Price/m2):
   id  Width_m  Area_m2  Floors  Price_Per_m2
0   1     14.0    276.0     1.0         28.95
1   2      5.0     96.0     1.0         82.19
2   3     14.5    333.0     2.0         60.06
3   4      NaN   6000.0     1.0         15.83
4   5     23.5   2277.0     1.0         25.47
5   6      NaN     86.0     4.0         84.88
6   7      4.0     76.0     4.0         98.42
7   8      5.0     85.0     3.0         70.59
8   9      5.0     50.0     2.0        106.00
9  10      NaN     24.0     3.0        127.08


Lọc Thiếu Price_Per_m2, Thiếu District, Giá quá rẻ (< 500 triệu), Width_m, trùng lặp, Điền khuyết thông minh: Tự động tính Median số phòng ngủ/toilet của từng Quận để điền vào chỗ trống của Quận đó.

In [7]:
import pandas as pd
import numpy as np

# --- CẤU HÌNH ---
# Input là file đã tính toán xong ở bước trước
INPUT_FILE = 'batdongsan_data_CLEAN_FULL_CALCULATED_CAITIEN.csv'
OUTPUT_FILE = 'batdongsan_data_READY_FOR_MODEL_CAITIEN.csv'

try:
    print("⏳ Đang đọc dữ liệu...")
    df = pd.read_csv(INPUT_FILE)
    n_original = len(df)
    print(f"--> Tổng số dòng ban đầu: {n_original}")

    # ================= BƯỚC 1: THANH LỌC DỮ LIỆU (FILTERING) =================

    # 1.1 Xóa các dòng bị thiếu thông tin CHÍ MẠNG (Critical Missing)
    # Thêm 'Area_m2' vào đây để loại bỏ ngay 49 dòng bị lỗi diện tích
    critical_cols = ['Price_Per_m2', 'District', 'Width_m', 'Area_m2']
    df = df.dropna(subset=critical_cols)

    # Lọc thêm: Đảm bảo đơn giá phải > 0 (tránh lỗi chia cho 0 còn sót)
    df = df[df['Price_Per_m2'] > 0]

    print(f"- Sau khi xóa thiếu Giá/Quận/Chiều ngang/Diện tích: {len(df)}")

    # 1.2 Lọc theo Giá trị (Price Billion)
    # Yêu cầu: Giá quá rẻ (< 500 triệu hay 0.5 tỷ) -> XÓA
    df = df[df['Price_Billion'] >= 0.5]
    print(f"- Sau khi xóa nhà < 500 triệu: {len(df)}")

    # 1.3 Xử lý trùng lặp (Duplicates)
    # Logic: Nếu Tiêu đề, Giá, Diện tích và Quận giống hệt nhau -> Coi là spam -> XÓA
    df = df.drop_duplicates(subset=['Title', 'Price_Billion', 'Area_m2', 'District'])
    print(f"- Sau khi xóa tin Spam trùng lặp: {len(df)}")

    # *LƯU Ý: Các điều kiện Diện tích <10, >500, Giá ảo >800 -> KHÔNG LỌC (Theo yêu cầu)*

    # ================= BƯỚC 2: ĐIỀN KHUYẾT THÔNG MINH (GROUP IMPUTATION) =================
    # Yêu cầu: Dùng Median (Trung vị) của từng QUẬN để điền vào Bedrooms/Toilets bị thiếu

    print("\n⏳ Đang tính toán Median theo từng Quận để điền khuyết...")

    # Hàm điền median theo nhóm
    def fill_na_with_district_median(df, target_col, group_col='District'):
        # Tính median cho từng nhóm (Quận)
        median_series = df.groupby(group_col)[target_col].transform('median')

        # Điền các ô NaN bằng giá trị median vừa tính
        df[target_col] = df[target_col].fillna(median_series)

        # "Chữa cháy": Nếu cả Quận đó không có dữ liệu nào (hiếm), dùng Median toàn thành phố
        global_median = df[target_col].median()
        df[target_col] = df[target_col].fillna(global_median)

        # Làm tròn số (vì phòng ngủ/toilet phải là số nguyên)
        df[target_col] = df[target_col].round()
        return df

    # Thực hiện điền cho Bedrooms & Toilets
    df = fill_na_with_district_median(df, 'Bedrooms')
    df = fill_na_with_district_median(df, 'Toilets')

    # ================= BƯỚC 3: KIỂM TRA CUỐI CÙNG & LƯU FILE =================

    # Check lần cuối xem còn sót ô Null nào không
    if df.isnull().values.any():
        print("\n⚠️ CẢNH BÁO: Vẫn còn dữ liệu bị thiếu! Hãy kiểm tra lại code.")
        print(df.isnull().sum()[df.isnull().sum() > 0])
    else:
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

        print("-" * 50)
        print(f"✅ HOÀN TẤT! DỮ LIỆU SẠCH 100% (NO MISSING VALUES)")
        print(f"📁 File kết quả: {OUTPUT_FILE}")
        print(f"📊 Số lượng dòng còn lại: {len(df)} (Giữ lại {round(len(df)/n_original*100, 1)}% dữ liệu gốc)")
        print("-" * 50)

        # In mẫu kiểm tra Quận Gò Vấp để báo cáo tính năng điền Median
        print("\n[Demo Báo Cáo] Kiểm tra dữ liệu Quận Gò Vấp sau khi điền Median:")
        sample = df[df['District'].str.contains('Gò Vấp', na=False, case=False)]
        if not sample.empty:
            print(sample[['District', 'Bedrooms', 'Toilets']].head(5).to_string())
        else:
            print("(Không có dữ liệu Gò Vấp để demo, in 5 dòng đầu bất kỳ)")
            print(df[['District', 'Bedrooms', 'Toilets']].head(5).to_string())

except Exception as e:
    print(f"❌ Lỗi: {e}")

⏳ Đang đọc dữ liệu...
--> Tổng số dòng ban đầu: 31176
- Sau khi xóa thiếu Giá/Quận/Chiều ngang/Diện tích: 23501
- Sau khi xóa nhà < 500 triệu: 23356
- Sau khi xóa tin Spam trùng lặp: 23353

⏳ Đang tính toán Median theo từng Quận để điền khuyết...
--------------------------------------------------
✅ HOÀN TẤT! DỮ LIỆU SẠCH 100% (NO MISSING VALUES)
📁 File kết quả: batdongsan_data_READY_FOR_MODEL_CAITIEN.csv
📊 Số lượng dòng còn lại: 23353 (Giữ lại 74.9% dữ liệu gốc)
--------------------------------------------------

[Demo Báo Cáo] Kiểm tra dữ liệu Quận Gò Vấp sau khi điền Median:
   District  Bedrooms  Toilets
6    Gò Vấp       4.0      4.0
11   Gò Vấp       2.0      2.0
12   Gò Vấp       4.0      5.0
17   Gò Vấp       2.0      2.0
24   Gò Vấp       4.0      4.0


Từ title + description => phường(ward)

In [3]:
import pandas as pd
import re

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_READY_FOR_MODEL_CAITIEN.csv'
OUTPUT_FILE = 'batdongsan_data_WITH_WARD.csv'

# 1. TỪ ĐIỂN PHƯỜNG/XÃ (Đã tách riêng Q2, Q9, Thủ Đức để chuẩn xác hơn)
hcm_wards_advanced = {
    'Quận 1': ['Bến Nghé', 'Bến Thành', 'Cô Giang', 'Cầu Kho', 'Cầu Ông Lãnh', 'Đa Kao', 'Nguyễn Cư Trinh', 'Nguyễn Thái Bình', 'Phạm Ngũ Lão', 'Tân Định'],
    'Quận 3': ['1', '2', '3', '4', '5', '9', '10', '11', '12', '14', 'Võ Thị Sáu', '6', '7', '8'],
    'Quận 4': ['1', '2', '3', '4', '6', '8', '9', '10', '13', '14', '15', '16', '18', '12'],
    'Quận 5': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    'Quận 6': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'],
    'Quận 7': ['Bình Thuận', 'Phú Mỹ', 'Phú Thuận', 'Tân Hưng', 'Tân Kiểng', 'Tân Phong', 'Tân Phú', 'Tân Quy', 'Tân Thuận Đông', 'Tân Thuận Tây'],
    'Quận 8': ['1', '2', '3', '4', '5', '6', '8', '7', '9', '10', '11', '12', '13', '14', '15', '16'],
    'Quận 10': ['1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '3'],
    'Quận 11': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    'Quận 12': ['An Phú Đông', 'Đông Hưng Thuận', 'Hiệp Thành', 'Tân Chánh Hiệp', 'Tân Hưng Thuận', 'Tân Thới Hiệp', 'Tân Thới Nhất', 'Thạnh Lộc', 'Thạnh Xuân', 'Thới An', 'Trung Mỹ Tây'],
    'Bình Thạnh': ['1', '2', '3', '5', '6', '7', '11', '12', '13', '14', '15', '17', '19', '21', '22', '24', '25', '26', '27', '28'],
    'Gò Vấp': ['1', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17'],
    'Phú Nhuận': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '13', '15', '17'],
    'Tân Bình': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    'Tân Phú': ['Hiệp Tân', 'Hòa Thạnh', 'Phú Thọ Hòa', 'Phú Thạnh', 'Phú Trung', 'Sơn Kỳ', 'Tân Quý', 'Tân Sơn Nhì', 'Tân Thành', 'Tân Thới Hòa', 'Tây Thạnh'],
    'Bình Tân': ['An Lạc', 'An Lạc A', 'Bình Hưng Hòa', 'Bình Hưng Hòa A', 'Bình Hưng Hòa B', 'Bình Trị Đông', 'Bình Trị Đông A', 'Bình Trị Đông B', 'Tân Tạo', 'Tân Tạo A'],

    # Khu vực TP Thủ Đức (Tách ra để chính xác)
    'Thủ Đức': ['Bình Chiểu', 'Bình Thọ', 'Hiệp Bình Chánh', 'Hiệp Bình Phước', 'Linh Chiểu', 'Linh Đông', 'Linh Tây', 'Linh Trung', 'Linh Xuân', 'Tam Bình', 'Tam Phú', 'Trường Thọ'],
    'Quận 2': ['An Khánh', 'An Lợi Đông', 'An Phú', 'Bình An', 'Bình Khánh', 'Bình Trưng Đông', 'Bình Trưng Tây', 'Cát Lái', 'Thạnh Mỹ Lợi', 'Thảo Điền', 'Thủ Thiêm'],
    'Quận 9': ['Hiệp Phú', 'Long Bình', 'Long Phước', 'Long Thạnh Mỹ', 'Long Trường', 'Phú Hữu', 'Phước Bình', 'Phước Long A', 'Phước Long B', 'Tân Phú', 'Tăng Nhơn Phú A', 'Tăng Nhơn Phú B'],

    # Huyện
    'Củ Chi': ['An Nhơn Tây', 'Bình Mỹ', 'Hòa Phú', 'Nhuận Đức', 'Phạm Văn Cội', 'Phú Hòa Đông', 'Phú Mỹ Hưng', 'Phước Hiệp', 'Phước Thạnh', 'Phước Vĩnh An', 'Tân An Hội', 'Tân Phú Trung', 'Tân Thạnh Đông', 'Tân Thạnh Tây', 'Tân Thông Hội', 'Thái Mỹ', 'Trung An', 'Trung Lập Hạ', 'Trung Lập Thượng', 'Củ Chi', 'Thị trấn Củ Chi'],
    'Hóc Môn': ['Bà Điểm', 'Đông Thạnh', 'Nhị Bình', 'Tân Hiệp', 'Tân Thới Nhì', 'Tân Xuân', 'Thới Tam Thôn', 'Trung Chánh', 'Xuân Thới Đông', 'Xuân Thới Sơn', 'Xuân Thới Thượng', 'Hóc Môn', 'Thị trấn Hóc Môn'],
    'Bình Chánh': ['An Phú Tây', 'Bình Chánh', 'Bình Hưng', 'Bình Lợi', 'Đa Phước', 'Hưng Long', 'Lê Minh Xuân', 'Phạm Văn Hai', 'Phong Phú', 'Quy Đức', 'Tân Kiên', 'Tân Nhựt', 'Tân Quý Tây', 'Vĩnh Lộc A', 'Vĩnh Lộc B', 'Tân Túc', 'Thị trấn Tân Túc'],
    'Nhà Bè': ['Hiệp Phước', 'Long Thới', 'Nhơn Đức', 'Phú Xuân', 'Phước Kiển', 'Phước Lộc', 'Nhà Bè', 'Thị trấn Nhà Bè'],
    'Cần Giờ': ['An Thới Đông', 'Bình Khánh', 'Long Hòa', 'Lý Nhơn', 'Tam Thôn Hiệp', 'Thạnh An', 'Cần Thạnh', 'Thị trấn Cần Thạnh']
}

# 2. MAPPING SÁP NHẬP (QUAN TRỌNG: MAP MỚI VỀ CŨ)
# Cấu trúc: ('Quận', 'Tên Mới'): 'Tên Cũ Đại Diện'
merged_ward_mapping = {
    # Gò Vấp: Hạnh Thông = P1 + P3 (Chọn P1)
    ('Gò Vấp', 'Hạnh Thông'): '1',
    ('Gò Vấp', 'Phường Hạnh Thông'): '1',

    # Quận 3: Võ Thị Sáu = P6 + P7 + P8 (Chọn P6)
    ('Quận 3', 'Võ Thị Sáu'): '6',

    # Quận 2 (Thủ Đức): An Khánh mới = Bình An + Bình Khánh (Chọn Bình An)
    ('Quận 2', 'An Khánh'): 'Bình An',
    ('Thủ Đức', 'An Khánh'): 'Bình An',

    # Quận 4: P12 nhập vào P13
    ('Quận 4', '12'): '13',

    # Quận 5: P15 nhập vào P12
    ('Quận 5', '15'): '12',

    # Quận 10: P3 nhập vào P2
    ('Quận 10', '3'): '2',

    # Phú Nhuận: P12 -> P11, P14 -> P13 (Ví dụ)
    ('Phú Nhuận', '12'): '11',
    ('Phú Nhuận', '14'): '13'
}

def normalize_district_key(district_raw):
    if pd.isna(district_raw): return None
    d = str(district_raw).strip()
    if d in ['Q2', 'Q.2', 'Quận 2', 'District 2']: return 'Quận 2'
    if d in ['Q9', 'Q.9', 'Quận 9', 'District 9']: return 'Quận 9'
    if d in ['Thủ Đức', 'Q.Thủ Đức', 'TP Thủ Đức', 'Thành phố Thủ Đức']: return 'Thủ Đức'
    if d in hcm_wards_advanced: return d
    if d.startswith('Q') and d[1:].isdigit(): return f"Quận {d[1:]}"
    if d.startswith('Q.') and d[2:].isdigit(): return f"Quận {d[2:]}"
    return d

def extract_ward(row):
    district_key = normalize_district_key(row['District'])
    if not district_key: return None

    # Gộp text để tìm kiếm
    text_search = (str(row.get('Title', '')) + ' ' + str(row.get('Description', '')) + ' ' + str(row.get('Address', ''))).lower()

    # --- ƯU TIÊN 1: CHECK MAPPING SÁP NHẬP (TÊN MỚI) ---
    for (dist, new_ward), old_ward_target in merged_ward_mapping.items():
        # Kiểm tra đúng quận (hoặc quận thuộc nhóm Thủ Đức)
        is_match_district = (dist == district_key) or \
                            (district_key == 'Thủ Đức' and dist in ['Quận 2', 'Quận 9', 'Thủ Đức'])

        if is_match_district:
            # Nếu tìm thấy tên phường mới (VD: "Hạnh Thông") trong văn bản
            if new_ward.lower() in text_search:
                return old_ward_target # Trả về tên cũ (VD: "1")

    # --- ƯU TIÊN 2: TÌM KIẾM BÌNH THƯỜNG ---
    possible_wards = []
    if district_key in hcm_wards_advanced:
        possible_wards = hcm_wards_advanced[district_key]
    elif district_key in ['Thành phố Thủ Đức', 'TP. Thủ Đức']:
         # Nếu input chỉ ghi chung là TP Thủ Đức, ta tìm trong cả 3 quận cũ
         possible_wards = hcm_wards_advanced['Thủ Đức'] + hcm_wards_advanced['Quận 2'] + hcm_wards_advanced['Quận 9']
    else:
        return None

    # Tìm Phường Số (P1, F.1, Phường 1...)
    numeric_wards = [w for w in possible_wards if w.isdigit()]
    for ward in numeric_wards:
        # Regex thông minh: bắt p, f, phường + số + không có số đằng sau
        pattern = r'(?:phường|p|f)[\.\s]*0?' + ward + r'(?!\d)'
        if re.search(pattern, text_search):
            return ward

    # Tìm Phường Chữ (Tân Định, Đa Kao...)
    named_wards = [w for w in possible_wards if not w.isdigit()]
    named_wards.sort(key=len, reverse=True) # Tìm tên dài trước

    for ward in named_wards:
        if ward.lower() in text_search:
            return ward

    return None

# --- CHẠY CHƯƠNG TRÌNH ---
print("Đang xử lý trích xuất Phường (Advanced Mapping)...")
try:
    df = pd.read_csv(INPUT_FILE)

    df['Ward'] = df.apply(extract_ward, axis=1)

    # Thống kê
    total = len(df)
    found = df['Ward'].notna().sum()
    print(f"Tổng số dòng: {total}")
    print(f"Tìm thấy Phường: {found} ({found/total*100:.2f}%)")
    print("-" * 30)
    print(df[['District', 'Ward']].head(10))

    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"Đã lưu file kết quả: {OUTPUT_FILE}")

except Exception as e:
    print(f"Lỗi: {e}")

Đang xử lý trích xuất Phường (Advanced Mapping)...
Tổng số dòng: 23353
Tìm thấy Phường: 16533 (70.80%)
------------------------------
  District             Ward
0  Hóc Môn    Xuân Thới Sơn
1  Quận 12       Hiệp Thành
2  Quận 12    Tân Thới Nhất
3  Hóc Môn       Đông Thạnh
4   Gò Vấp                1
5   Quận 2             None
6   Quận 7         Tân Hưng
7   Quận 9  Tăng Nhơn Phú A
8   Gò Vấp               14
9   Gò Vấp             None
Đã lưu file kết quả: batdongsan_data_WITH_WARD.csv


Lọc chỉ lấy những cột có trường Ward không null

In [1]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_WITH_WARD.csv'        # File vừa chạy xong ở bước trước
OUTPUT_FILE = 'batdongsan_data_FINAL_VALID_WARD.csv' # File SẠCH SẼ (Chỉ chứa data có Ward)

try:
    print(f"⏳ Đang đọc file '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)

    initial_count = len(df)

    # 1. LỌC DỮ LIỆU
    # Giữ lại các dòng mà cột 'Ward' KHÔNG bị rỗng (notna)
    df_valid = df[df['Ward'].notna()]

    # 2. LƯU FILE
    df_valid.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    # 3. BÁO CÁO KẾT QUẢ
    valid_count = len(df_valid)
    dropped_count = initial_count - valid_count

    print("-" * 50)
    print(f"✅ ĐÃ XUẤT THÀNH CÔNG FILE: {OUTPUT_FILE}")
    print("-" * 50)
    print(f"📊 Tổng số dòng ban đầu: {initial_count}")
    print(f"🟢 Số dòng HỢP LỆ (Có Phường): {valid_count} dòng")
    print(f"🔴 Số dòng BỊ LOẠI (Null Phường): {dropped_count} dòng")
    print("-" * 50)

    # In mẫu để kiểm tra
    print("\nMẫu 5 dòng dữ liệu chuẩn đã lọc:")
    print(df_valid[['District', 'Ward', 'Title']].head(5).to_string())

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{INPUT_FILE}'. Bạn hãy chắc chắn đã chạy code trích xuất Phường ở bước trước.")
except KeyError:
    print("❌ Lỗi: File không có cột 'Ward'. Hãy kiểm tra lại.")

⏳ Đang đọc file 'batdongsan_data_WITH_WARD.csv'...
--------------------------------------------------
✅ ĐÃ XUẤT THÀNH CÔNG FILE: batdongsan_data_FINAL_VALID_WARD.csv
--------------------------------------------------
📊 Tổng số dòng ban đầu: 23353
🟢 Số dòng HỢP LỆ (Có Phường): 16533 dòng
🔴 Số dòng BỊ LOẠI (Null Phường): 6820 dòng
--------------------------------------------------

Mẫu 5 dòng dữ liệu chuẩn đã lọc:
  District           Ward                                                                   Title
0  Hóc Môn  Xuân Thới Sơn       SIÊU PHẨM GÓC 2 MẶT TIỀN – TRƯƠNG THỊ NHƯ, XUÂN THỚI SƠN, HÓC MÔN
1  Quận 12     Hiệp Thành   BÁN GẤP NHÀ MẶT TIỀN HIỆP THÀNH 17 – TRUNG TÂM QUẬN 12 – GIÁ CHỐT MỀM
2  Quận 12  Tân Thới Nhất  HOT – HÀNG HIẾM GIỮA QUẬN 12 BÁN XƯỞNG HẺM XE TẢI 8M, P. TÂN THỚI NHẤT
3  Hóc Môn     Đông Thạnh            Bán nhà mặt tiền 23x99m giá 58 tỷ Đường Nhị Bình 15
4   Gò Vấp              1         Nhanh mới kịp – NHÀ ĐẸP Ô TÔ ĐỖ CỬA 76M² 4 TẦNG – NHỈNH 7 TỶ

Lọc ra các trường cần thiết

In [3]:
import pandas as pd

# --- CẤU HÌNH ---
INPUT_FILE = 'batdongsan_data_FINAL_VALID_WARD.csv'  # File đã có đủ Phường
OUTPUT_FILE = 'batdongsan_data_FILTERED_12_COLS.csv' # File kết quả

try:
    print(f"⏳ Đang đọc file '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)

    # 1. XỬ LÝ CỘT "IS HEM" (Tạo từ Is_MatTien)
    # Logic: Nếu Is_MatTien = 0 thì Is Hem = 1
    if 'Is Hem' not in df.columns:
        print("- Đang tạo cột 'Is Hem'...")
        df['Is Hem'] = df['Is_MatTien'].apply(lambda x: 1 if x == 0 else 0)

    # 2. XỬ LÝ CỘT "POST TIME"
    # Trong code cũ tên là 'Post_Time' (có gạch dưới), nếu bạn muốn tên là 'Post Time' (có dấu cách) thì đổi lại
    if 'Post_Time' in df.columns:
        df.rename(columns={'Post_Time': 'Post Time'}, inplace=True)

    # 3. DANH SÁCH 12 CỘT BẠN YÊU CẦU
    target_cols = [
        'Price_Billion',
        'Price_Per_m2',
        'Area_m2',
        'District',
        'Ward',
        'Bedrooms',
        'Is_MatTien',
        'Width_m',
        'Floors',
        'Is Hem',
        'Post Time',
        'Toilets'
    ]

    # 4. THỰC HIỆN LỌC VÀ LƯU FILE
    # Chỉ lấy đúng các cột trong danh sách
    df_final = df[target_cols]

    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("-" * 50)
    print(f"✅ ĐÃ XONG! File chỉ chứa đúng 12 cột: {OUTPUT_FILE}")
    print("-" * 50)
    print(df_final.head().to_string())

except KeyError as e:
    print(f"❌ Lỗi: Không tìm thấy cột {e} trong file gốc. Hãy kiểm tra lại tên cột.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file '{INPUT_FILE}'")
except Exception as e:
    print(f"❌ Lỗi: {e}")

⏳ Đang đọc file 'batdongsan_data_FINAL_VALID_WARD.csv'...
- Đang tạo cột 'Is Hem'...
--------------------------------------------------
✅ ĐÃ XONG! File chỉ chứa đúng 12 cột: batdongsan_data_FILTERED_12_COLS.csv
--------------------------------------------------
   Price_Billion  Price_Per_m2  Area_m2 District           Ward  Bedrooms  Is_MatTien  Width_m  Floors  Is Hem     Post Time  Toilets
0           7.99         28.95    276.0  Hóc Môn  Xuân Thới Sơn       1.0           1     14.0     1.0       0  5 ngày trước      1.0
1           7.89         82.19     96.0  Quận 12     Hiệp Thành       1.0           1      5.0     1.0       0  5 ngày trước      1.0
2          20.00         60.06    333.0  Quận 12  Tân Thới Nhất       2.0           0     14.5     2.0       1  5 ngày trước      2.0
3          58.00         25.47   2277.0  Hóc Môn     Đông Thạnh       2.0           1     23.5     1.0       0  5 ngày trước      2.0
4           7.48         98.42     76.0   Gò Vấp              1     